In [176]:
import pandas as pd

In [177]:
import json
import pprint as pprint
import numpy as np

In [178]:
with open('./slice.json', 'r') as f:
    datastore = json.load(f)

In [179]:
sample_playlists = datastore['playlists']

In [180]:
processed_data = {}

In [181]:
for playlist in sample_playlists:
    
    cumulative_track_features = {}    
    # go through each track and get the features
    for track in playlist['tracks']:
        
        if track['track_features'] is None:
            print("skipping track")
            continue
            
        for feature in track['track_features']:

            if feature in ['type', 'id', 'uri', 'track_href', 'analysis_url']:
                continue # we skip these since they're non-numeric
            
            # if we haven't seen the feature, init a list
            if not feature in cumulative_track_features:
                cumulative_track_features[feature] = []
            
            cumulative_track_features[feature].append(track['track_features'][feature])
    
    if cumulative_track_features == {}:
        print("skip playlist")
        continue # skip, no features found 
        
    playlist_track_features = {}
    for feature in cumulative_track_features:
        playlist_track_features[feature] = sum(cumulative_track_features[feature])/len(cumulative_track_features[feature])
    
    processed_data[playlist['name']] = playlist_track_features

skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping track
skipping t

In [182]:
feature_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
playlist_names = []
data_matrix = []
for playlist in processed_data:
    playlist_names.append(playlist)
    row = []
    data = processed_data[playlist]
    print(data)
    for i in range(len(feature_keys)):
        row.append(data[feature_keys[i]])
    data_matrix.append(row)

{'danceability': 0.6555600000000001, 'energy': 0.7679999999999998, 'key': 5.78, 'loudness': -10.597940000000001, 'mode': 0.52, 'speechiness': 0.05971600000000001, 'acousticness': 0.04614114, 'instrumentalness': 0.8477756, 'liveness': 0.17186599999999996, 'valence': 0.2835640000000001, 'tempo': 130.7317, 'duration_ms': 412508.18, 'time_signature': 3.96}
{'danceability': 0.4789000000000002, 'energy': 0.5485580000000002, 'key': 5.1, 'loudness': -8.702, 'mode': 0.64, 'speechiness': 0.07517599999999999, 'acousticness': 0.31143990739999994, 'instrumentalness': 0.2338944624, 'liveness': 0.21795200000000003, 'valence': 0.26664400000000005, 'tempo': 114.12292000000001, 'duration_ms': 281206.94, 'time_signature': 3.88}
{'danceability': 0.356446, 'energy': 0.6582200000000001, 'key': 4.86, 'loudness': -8.697260000000002, 'mode': 0.8, 'speechiness': 0.042547999999999975, 'acousticness': 0.1727694576, 'instrumentalness': 0.5945090999999999, 'liveness': 0.207958, 'valence': 0.2575400000000001, 'tempo

{'danceability': 0.5762688172043012, 'energy': 0.8691774193548388, 'key': 5.002688172043011, 'loudness': -5.990494623655917, 'mode': 0.7876344086021505, 'speechiness': 0.05917258064516131, 'acousticness': 0.1567033322311828, 'instrumentalness': 0.0778293757526882, 'liveness': 0.20251827956989255, 'valence': 0.874438172043011, 'tempo': 130.83298387096778, 'duration_ms': 200733.91129032258, 'time_signature': 3.9919354838709675}
{'danceability': 0.6182222222222221, 'energy': 0.6150555555555555, 'key': 5.222222222222222, 'loudness': -7.514222222222224, 'mode': 0.3888888888888889, 'speechiness': 0.12287222222222224, 'acousticness': 0.22953833333333337, 'instrumentalness': 0.064352825, 'liveness': 0.20971111111111113, 'valence': 0.42270555555555556, 'tempo': 112.47522222222223, 'duration_ms': 251630.27777777778, 'time_signature': 3.9444444444444446}
{'danceability': 0.5207341772151899, 'energy': 0.7006360759493668, 'key': 5.3354430379746836, 'loudness': -5.4759810126582265, 'mode': 0.6392405

In [183]:
X = np.array(data_matrix)


In [184]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

In [185]:
model_knn.fit(X)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
                 radius=1.0)

In [186]:
distances, indices = model_knn.kneighbors(X)

In [187]:
print("Similar Playlists for {}".format(playlist_names[0]))
indices_for_playlist_0_neighbors = indices[0]
distances_for_playlist_0_neighbors = distances[0]
for index in indices_for_playlist_0_neighbors:
    print("Name: {}".format(playlist_names[index]))

Similar Playlists for r/Techno | Top weekly posts
Name: r/Techno | Top weekly posts
Name: Techno Youth
Name: Epic Workout
Name: =] DARK HERESY [=
Name: Dunkles Geballer
Name: Strictly Lo-Fi
Name: Buugeng
Name: r/FutureSynth | Top weekly posts
Name: Chilled Indie / Unsigned
Name: Constellations of Summer
